
# Smart environment Hotspot & Risk Prediction using Traffy Fondue

This notebook is a **full project pipeline** template for a environment-related data project using **Traffy Fondue** data.

- Uses Traffy Fondue complaints data
- Adds external data (you plug in your own API/scraping)
- Includes:
  - Data Engineering (cleaning + aggregation)
  - Machine Learning (predict environment complaints)
  - Visualization (time trends, optional map)


## Import Libraries

In [148]:

# === Config & Imports ===

# If running in a fresh environment, you may need to install:
# !pip install pandas numpy scikit-learn matplotlib seaborn requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

import json
import time
import requests
import re
from bs4 import BeautifulSoup

import requests



from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Optional: for geospatial visualization
try:
    import geopandas as gpd
except ImportError:
    gpd = None
    print("geopandas not installed. Geospatial choropleth will be skipped unless installed.")

pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid")


## 1. Load Traffy Fondue Data

In [149]:
df = pd.read_csv('bangkok_traffy.csv')
print("Raw Traffy shape:", df.shape)
df.head()


Raw Traffy shape: (787026, 16)


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


## 2. Data Cleansing

In [150]:
import pandas as pd
import numpy as np

# --- Read CSV ---
df = pd.read_csv("bangkok_traffy.csv")

# --- Keep only selected columns ---
keep_cols = [
    "ticket_id", "type", "coords", "comment", "organization",
    "district", "province", "timestamp", "state", "last_activity",
    "subdistrict"
]
df = df[keep_cols]

# ================================
# DATA CLEANING
# ================================

# First, replace empty strings and "{}" with NaN BEFORE converting to string
df = df.replace(["", "{}", "nan"], np.nan)

# Now clean text columns (but don't convert NaN to string)
text_cols = ["subdistrict", "district", "province", "type", "comment", "organization"]

for col in text_cols:
    # Only strip non-null values
    df[col] = df[col].str.strip()

# Remove rows with missing comments
df = df[df["comment"].notna()].copy()

# --- Remove rows with missing required fields ---
df = df[
    df["district"].notna() &
    df["province"].notna() &
    df["type"].notna()
].copy()

# ================================
# FILTER ONLY กรุงเทพมหานคร + เสร็จสิ้น
# ================================

df = df[
    (df["province"] == "กรุงเทพมหานคร") &
    (df["state"] == "เสร็จสิ้น")
].copy()

print(f"\nRows after province filter: {len(df)}")

# ================================
# REMOVE ALL ROWS WITH ANY NaN VALUES
# ================================
print(f"Rows before removing NaN: {len(df)}")
print("\nMissing values before dropna:")
print(df.isna().sum())

df = df.dropna()

print(f"\nRows after removing NaN: {len(df)}")

# ================================
# FINAL OUTPUT
# ================================
print("\n=== CLEANED DATA (HEAD) ===")
print(df.head())

print("\n=== MISSING VALUES SUMMARY ===")
print(df.isna().sum())

print(f"\n=== FINAL ROW COUNT: {df.shape[0]} ===")

# # Optional: Save cleaned data
df.to_csv("bangkok_traffy_cleaned.csv", index=False)
# print("\n✅ Cleaned data saved to 'bangkok_traffy_cleaned.csv'")


Rows after province filter: 439679
Rows before removing NaN: 439679

Missing values before dropna:
ticket_id        0
type             0
coords           0
comment          0
organization     0
district         0
province         0
timestamp        0
state            0
last_activity    0
subdistrict      0
dtype: int64

Rows after removing NaN: 439679

=== CLEANED DATA (HEAD) ===
     ticket_id                 type              coords  \
1  2021-CGPMUN  {น้ำท่วม,ร้องเรียน}  100.66709,13.67891   
2  2021-7XATFA              {สะพาน}  100.52649,13.72060   
4  2021-DVEWYM        {น้ำท่วม,ถนน}  100.59165,13.82280   
7  2021-8N9ZP8          {ความสะอาด}  100.64690,13.67083   
9  2021-8BTWZB        {ท่อระบายน้ำ}  100.65440,13.68158   

                                             comment  \
1  น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...   
2    สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12   
4  ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...   
7                  คนเอาขยะมาทิ้งจนกล

## 3. Scrape District Area Data

In [151]:
PROVINCES = {
    "กรุงเทพมหานคร": "Bangkok.htm",
}

BASE_URL = "https://e-report.energy.go.th/area/"

def fetch_html(url):
    """Fetch HTML with proper Thai encoding"""
    resp = requests.get(url)
    
    for encoding in ['tis-620', 'windows-874', 'cp874']:
        try:
            resp.encoding = encoding
            text = resp.text
            if 'กรุงเทพ' in text or 'เนื้อที่' in text:
                print(f"  → Using encoding: {encoding}")
                return text
        except:
            continue
    
    resp.encoding = resp.apparent_encoding
    print(f"  → Using auto-detected encoding: {resp.apparent_encoding}")
    return resp.text

def parse_province_table(html, province_name):
    """Parse HTML table and extract area data"""
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        print(f"[WARN] No table found for {province_name}")
        return []

    header_cells = table.find("tr").find_all(["td", "th"])
    headers = [h.get_text(strip=True) for h in header_cells]
    print(f"  → Headers: {headers[:3]}...")

    area_idx = None
    for i, h in enumerate(headers):
        if "เนื้อที่" in h or "พื้นที่" in h:
            area_idx = i
            break

    if area_idx is None:
        print(f"[WARN] No area column for {province_name}, skipping")
        return []

    name_idx = 1
    rows_data = []
    skipped_count = 0
    mueang_renamed = 0
    
    for tr in table.find_all("tr")[1:]:
        cols = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(cols) <= area_idx:
            continue
        district = cols[name_idx]
        area = cols[area_idx]

        if district == "" or area == "" or "รวม" in district.lower():
            continue
        
        # Bangkok filter: only "เขต", skip "แขวง"
        if province_name == "กรุงเทพมหานคร":
            if district.startswith("แขวง"):
                skipped_count += 1
                continue
            if not district.startswith("เขต"):
                continue
        
        # Handle "เมือง" districts - append province name
        # Remove common prefixes first to get clean province name
        if district == "เมือง" or district == "อำเภอเมือง":
            # Get short province name (without common prefixes)
            province_short = province_name.replace("จังหวัด", "").strip()
            district = f"เมือง{province_short}"
            mueang_renamed += 1
            print(f"  → Renamed 'เมือง' to '{district}'")

        rows_data.append({
            "province": province_name,
            "district": district,
            "area_km2": area
        })
    
    if province_name == "กรุงเทพมหานคร" and skipped_count > 0:
        print(f"  → Skipped {skipped_count} แขวง (sub-districts)")
    
    if mueang_renamed > 0:
        print(f"  → Renamed {mueang_renamed} 'เมือง' districts")

    return rows_data

# Scrape all provinces
area_rows = []

for prov_th, page in PROVINCES.items():
    url = BASE_URL + page
    print(f"\nScraping {prov_th} ... {url}")
    html = fetch_html(url)
    rows = parse_province_table(html, prov_th)
    area_rows.extend(rows)
    print(f"  → Extracted {len(rows)} districts")

print(f"\n{'='*50}")
print(f"Total area records: {len(area_rows)}")


Scraping กรุงเทพมหานคร ... https://e-report.energy.go.th/area/Bangkok.htm
  → Using encoding: tis-620
  → Headers: ['ลำดับ', 'อำเภอ/กิ่งอำเภอ', 'เนื้อที่ (ตร.กม.)']...
  → Skipped 160 แขวง (sub-districts)
  → Extracted 50 districts

Total area records: 50


## 4. Fetch API Data (Population & Housing)

In [152]:

cookies = {
    "_ga_WPG9F4YRE3": "GS2.1.s1765033766$o1$g0$t1765033766$j60$l0$h0",
    "_ga": "GA1.1.1399864746.1765033766",
    "TS012f8194": "010214bde367a4be39fc7d733390be0576058f11ad2f47a6c577a3be26dc2c439a593fbc739960a059e82d87d4bf1e8caa2f6444df"
}

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

def call_dopa(url):
    """Call DOPA API"""
    r = requests.get(url, cookies=cookies, headers=headers)
    print("  STATUS:", r.status_code)
    print("  RAW (first 200 chars):", r.text[:200])

    try:
        return r.json()
    except:
        print(" JSON decode error")
        return None

# Config
begin = [6501, 6601, 6701,6801]
end   = [6512, 6612, 6712,6812]
cc    = [10]

population_file = "population_data.json"
housing_file = "housing_data.json"

# Load or create
try:
    with open(population_file, "r", encoding="utf-8") as f:
        population_data = json.load(f)
    print(f" - Loaded existing {population_file}")
except:
    population_data = []
    print(f" - Creating new {population_file}")

try:
    with open(housing_file, "r", encoding="utf-8") as f:
        housing_data = json.load(f)
    print(f" - Loaded existing {housing_file}")
except:
    housing_data = []
    print(f" - Creating new {housing_file}")

# Fetch data
for b, e in zip(begin, end):
    for province in cc:

        # Population API
        pop_url = (
            "https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php"
            f"?API=/api/statpophouse/v1/statpop/list"
            f"?action=43"
            f"&yymmBegin={b}"
            f"&yymmEnd={e}"
            f"&statType=0&statSubType=999&subType=99"
            f"&cc={province}"
        )

        print(f"\n--- Fetching POPULATION: yymm {b}-{e}, cc={province}")
        pop_data = call_dopa(pop_url)

        if pop_data:
            population_data.append({
                "begin": b,
                "end": e,
                "cc": province,
                "data": pop_data
            })

        time.sleep(0.5)

        # Housing API
        house_url = (
            "https://stat.bora.dopa.go.th/stat/statnew/connectSAPI/stat_forward.php"
            f"?API=/api/statpophouse/v1/stathouse/list"
            f"?action=33"
            f"&yymmBegin={b}"
            f"&yymmEnd={e}"
            f"&statType=0&statSubType=999&subType=99"
            f"&cc={province}"
        )

        print(f"--- Fetching HOUSING: yymm {b}-{e}, cc={province}")
        house_data = call_dopa(house_url)

        if house_data:
            housing_data.append({
                "begin": b,
                "end": e,
                "cc": province,
                "data": house_data
            })

        time.sleep(0.5)

# Save JSON files (for caching/debugging)
with open(population_file, "w", encoding="utf-8") as f:
    json.dump(population_data, f, indent=2, ensure_ascii=False)

with open(housing_file, "w", encoding="utf-8") as f:
    json.dump(housing_data, f, indent=2, ensure_ascii=False)

print(f"\n API data fetched and cached to JSON files")

 - Creating new population_data.json
 - Loaded existing housing_data.json

--- Fetching POPULATION: yymm 6501-6512, cc=10


  STATUS: 200
  RAW (first 200 chars): [{"lstrLevel":0,"lsregion":0,"lscc":10,"lsccDesc":"กรุงเทพมหานคร","lsrcode":"1001","lsrcodeDesc":"ท้องถิ่นเขตพระนคร","lsaa":0,"lsaaDesc":" ","lstt":0,"lsttDesc":" ","lsmm":0,"lsmmDesc":" ","lsyymm":65
--- Fetching HOUSING: yymm 6501-6512, cc=10
  STATUS: 200
  RAW (first 200 chars): [{"lstrLevel":0,"lsregion":0,"lscc":10,"lsccDesc":"กรุงเทพมหานคร","lsrcode":"1001","lsrcodeDesc":"ท้องถิ่นเขตพระนคร","lsaa":0,"lsaaDesc":" ","lstt":0,"lsttDesc":" ","lsmm":0,"lsmmDesc":" ","lsyymm":65

--- Fetching POPULATION: yymm 6601-6612, cc=10
  STATUS: 200
  RAW (first 200 chars): [{"lstrLevel":0,"lsregion":0,"lscc":10,"lsccDesc":"กรุงเทพมหานคร","lsrcode":"1001","lsrcodeDesc":"ท้องถิ่นเขตพระนคร","lsaa":0,"lsaaDesc":" ","lstt":0,"lsttDesc":" ","lsmm":0,"lsmmDesc":" ","lsyymm":66
--- Fetching HOUSING: yymm 6601-6612, cc=10
  STATUS: 200
  RAW (first 200 chars): [{"lstrLevel":0,"lsregion":0,"lscc":10,"lsccDesc":"กรุงเทพมหานคร","lsrcode":"1001","lsrcodeDesc":"ท้องถิ่

## 5. Build Lookup Dictionaries

In [153]:

def clean_province_name(name: str) -> str:
    """Clean province name by removing prefix"""
    if not isinstance(name, str):
        return name
    
    # Remove "จังหวัด" prefix
    clean = re.sub(r"^จังหวัด", "", name).strip()
    return clean



def clean_district_name(name: str) -> str:
    """Clean district name by removing prefixes"""
    if not isinstance(name, str):
        return name

    prefixes = [
        r"^อำเภอ",
        r"^ท้องถิ่นเขต",
        r"^เขต"
    ]

    clean = name.strip()
    for p in prefixes:
        clean = re.sub(p, "", clean).strip()

    return clean
# Build area dictionary from scraped data
area_dict = {}
for row in area_rows:
    province = row["province"]
    district = clean_district_name(row["district"])
    area_km2 = row["area_km2"]
    
    key = (province, district)
    area_dict[key] = area_km2

print(f"Area lookup: {len(area_dict)} records")

# Build population dictionary
population_dict = {}

for entry in population_data:
    for row in entry.get("data", []):
        
        district_raw = row.get("lsrcodeDesc", "")
        
        if "ท้องถิ่นเทศบาล" in district_raw:
            continue
        
        district_clean = clean_district_name(district_raw)
        
        lsyymm = str(row.get("lsyymm", ""))
        year = lsyymm[:2]
        month = lsyymm[2:]
        
        province_raw = row.get("lsccDesc", "")
        province = clean_province_name(province_raw)  # Clean province name
        total_pop = row.get("lssumtotTot", "")
        
        key = (province, district_clean, year, month)
        population_dict[key] = total_pop

print(f"Population lookup: {len(population_dict)} records")

Area lookup: 50 records
Population lookup: 2350 records


## 6. Combine All Scraped Data

In [158]:

def clean_province_name(name: str) -> str:
    """Clean province name by removing prefix"""
    if not isinstance(name, str):
        return name
    
    # Remove "จังหวัด" prefix
    clean = re.sub(r"^จังหวัด", "", name).strip()
    return clean

combined_rows = []

for entry in housing_data:
    for row in entry.get("data", []):
        
        district_raw = row.get("lsrcodeDesc", "")
        
        if "ท้องถิ่นเทศบาล" in district_raw:
            continue
        
        district_clean = clean_district_name(district_raw)
        
        lsyymm = str(row.get("lsyymm", ""))
        year = lsyymm[:2]
        month = lsyymm[2:]
        
        province_raw = row.get("lsccDesc", "")
        province = clean_province_name(province_raw)  # Clean province name
        housing = row.get("lssumnotTermDate", "")
        
        # Get matching data
        key_pop = (province, district_clean, year, month)
        key_landfill = (province, district_clean, year)
        key_area = (province, district_clean)
        
        total_population = population_dict.get(key_pop, "")
        area_km2 = area_dict.get(key_area, "")
        
        # Calculate densities
        population_density = ""
        housing_density = ""
        
        if area_km2 and area_km2 != "":
            try:
                area_float = float(area_km2)
                
                if total_population and total_population != "":
                    pop_float = float(total_population)
                    population_density = f"{pop_float / area_float:.2f}"
                
                if housing and housing != "":
                    house_float = float(housing)
                    housing_density = f"{house_float / area_float:.2f}"
            except (ValueError, ZeroDivisionError):
                pass
        
        combined_rows.append({
            "province": province,
            "district": district_clean,
            "total_population": total_population,
            "housing": housing,
            "year": year,
            "month": month,
            "area_km2": area_km2,
            "population_density": population_density,
            "housing_density": housing_density
        })

print(f"Combined {len(combined_rows)} records")

# Sort
combined_rows.sort(key=lambda x: (x["province"], x["district"], x["year"], x["month"]))
print(f"Sorted by province, district, year, month")

def escape_csv_field(field):
    """Escape CSV field"""
    field = str(field)
    if ',' in field or '"' in field or '\n' in field:
        field = '"' + field.replace('"', '""') + '"'
    return field

output_csv = "clean_combined_data.csv"

with open(output_csv, "w", encoding="utf-8-sig") as f:
    # Write header
    f.write("province,district,total_population,housing,year,month,area_km2,population_density,housing_density\n")
    
    # Write data rows
    for row in combined_rows:
        line = (
            f"{escape_csv_field(row['province'])},"
            f"{escape_csv_field(row['district'])},"
            f"{escape_csv_field(row['total_population'])},"
            f"{escape_csv_field(row['housing'])},"
            f"{escape_csv_field(row['year'])},"
            f"{escape_csv_field(row['month'])},"
            f"{escape_csv_field(row['area_km2'])},"
            f"{escape_csv_field(row['population_density'])},"
            f"{escape_csv_field(row['housing_density'])}\n"
        )
        f.write(line)

print(f"Saved to {output_csv}")
print(f"Total rows: {len(combined_rows)}")

Combined 18642 records
Sorted by province, district, year, month
Saved to clean_combined_data.csv
Total rows: 18642
